In [18]:
import os 
import numpy as np 
import polars as pl 
from src.funcs import UpperMPCVariablesManager, LowerMPCVariablesManager 
from src.upper_mpc import step_upper_level
from src.pumps_selector import circular_right_shift
from upper_mpc import step_upper_level, UMPCDataBuffer
import matplotlib.pyplot as plt 
import time 
import pandas as pd 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
data_path_wsl = "/home/alqua/data/pump_station_data/"
upper_mpc_data = pl.read_parquet(os.path.join(data_path_wsl, "sim_data/sim_data_1h_full.par"))
upper_mpc_data_pan = pl.read_parquet(os.path.join(data_path_wsl, "sim_data/sim_data_1h_full.par"))

In [ ]:
start_date = pd.to_datetime("2024-02-15 00:00:00+00:00")
end_date = pd.to_datetime("2024-02-25 08:00:00+00:00")

upper_mpc_data = upper_mpc_data.filter((pl.col("time") >= start_date) & 
                                       (pl.col("time") <= end_date)
                                       ).with_columns(pl.col("CO2Emission")/1000, pl.col("price")/1000)

upper_mpc_data = upper_mpc_data.with_columns(pl.col("time").dt.day().alias("day_index"))
day_sliced_df = upper_mpc_data.filter(pl.col("day_index") == 15)


In [55]:
umpc_opt_results = step_upper_level(horizon= 24, 
                                    prices_values = day_sliced_df["price"], 
                                    co2_progn_values = day_sliced_df["CO2Emission"], 
                                    inflow_values = day_sliced_df["inflow_kf"], 
                                    h_init = day_sliced_df["level"][0], 
                                    energy_init = 20, 
                                    Qout_init = 200)


In [56]:
umpc_opt_results

{'Qout': 592.2418794184464,
 'height': 199.9999983656802,
 'cum_energy': 31.67348681131324,
 'co2_progn': 1.0130000000000001,
 'da_price': 0.061130000000000004,
 'objective': 1569.836878527145}

In [57]:
umpc_data = UMPCDataBuffer()

In [58]:
umpc_data.initialize(umpc_opt_results)

In [59]:
umpc_data.data

{'Qout': [592.2418794184464],
 'height': [199.9999983656802],
 'co2_progn': [1.0130000000000001],
 'cum_energy': [31.67348681131324],
 'da_price': [0.061130000000000004],
 'objective': [1569.836878527145]}